In [1]:
%load_ext autoreload
%autoreload 2

import os

import tqdm
from rich import print
from rich.traceback import install

import datasets


os.environ["HYDRA_FULL_ERROR"] = "1"
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f9fa6dfe290>>

In [2]:
# ! pip install -e ../
# ! mamba install -c conda-forge itables pandas -y
# ! pip install pytorchvideo
import copy
import json
import os
import pathlib
import random
import sys
import time
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from dataclasses import dataclass
from enum import Enum
from math import floor
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import datasets

import numpy as np
import pandas as pd
import PIL
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import tqdm
from hydra_zen import builds, instantiate
from PIL import Image
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from rich import print
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataloader import default_collate
from torchvision.transforms import Compose, RandomCrop, Resize, ToTensor
from torchvision.transforms._transforms_video import CenterCropVideo
from traitlets import default
from transformers import CLIPModel, CLIPProcessor
import datasets
from tali_wit.data import (
    AnyModalSample,
    dataclass_collate,
    default_image_transforms,
    ModalityTypes,
    select_subtitles_between_timestamps,
    TALIDataset,
)
from tqdm.notebook import tqdm



/home/evolvingfungus/miniconda/envs/minimal-ml-template/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/evolvingfungus/miniconda/envs/minimal-ml-template/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
from tali_wit.decorators import configurable
from tali_wit.utils import get_logger, load_json, save_json
from tali_wit.models import ModalityConfig

logger = get_logger(__name__)


In [4]:
from tali_wit.data_plus import TALIBaseDemoTransform, TALIBaseTransformConfig


transform = TALIBaseDemoTransform(
    config=TALIBaseTransformConfig(
        root_filepath="/data/datasets/tali-wit-2-1-buckets/",
        modality_list=[
            ModalityTypes.wit_image.value,
            ModalityTypes.wit_caption.value,
            ModalityTypes.wit_title.value,
            ModalityTypes.wit_main_body.value,
            ModalityTypes.youtube_image.value,
            ModalityTypes.youtube_video.value,
            ModalityTypes.youtube_subtitles.value,
            ModalityTypes.youtube_audio.value,
            ModalityTypes.youtube_description.value,
        ],
        rng_seed=42,
        top_k_tali=10,
        image_size=224,
        num_video_frames=200,
        num_audio_frames=160000,
        clip_duration_in_seconds=10.0,
    )
)
train_dataset = datasets.load_from_disk("/home/evolvingfungus/forge/workspaces/tali-2-2/train-set")
train_dataset = train_dataset.with_transform(transform)
val_dataset = datasets.load_from_disk("/home/evolvingfungus/forge/workspaces/tali-2-2/val-set")
val_dataset = val_dataset.with_transform(transform)
test_dataset = datasets.load_from_disk("/home/evolvingfungus/forge/workspaces/tali-2-2/test-set")
test_dataset = test_dataset.with_transform(transform)
num_samples = 0

# for idx, sample in enumerate(tqdm(dataset)):
#     output = {key: value.shape if isinstance(value, torch.Tensor) else value for key, value in sample.items()}
#     print(output)
        
#     if idx >= num_samples:
#         break

In [5]:
from collections import defaultdict
from distutils.command.upload import upload
import gradio as gr

In [6]:

# {
#     'wit_idx': 502620,
#     'wikipedia_caption_image': torch.Size([3, 224, 224]),
#     'wikipedia_text': '<section_title> Station layout </section_title>',
#     'youtube_video_id': '6e7RO-o6u6w',
#     'youtube_content_video': torch.Size([10, 3, 224, 224]),
#     'youtube_content_audio': torch.Size([16000]),
#     'youtube_description_text': "<ysub> it's open somebody's gonna be building something there so the neighborhood 
# isn't is in change i don't think shintomicho has much of a personality when they took away the kabuki theater it 
# really did change  </ysub>"
# }
from collections import defaultdict
from distutils.command.upload import upload
import gradio as gr
import torchvision
import torchaudio

dataset_dict = {"train": train_dataset, "val": val_dataset, "test": test_dataset}

def update_length_options(set_name):
    max_idx = len(dataset_dict[set_name])
    return gr.update(minimum=0, maximum=max_idx, step=1)

def generate_caption_output(caption_dict):
    with gr.Column() as output:
        for language_key, language_captions in caption_dict.items():
            with gr.Tab(language_key):
                for caption_key, caption in language_captions.items():
                    gr.Textbox(value=caption, label=caption_key)
                
    return gr.update(children=[output])

def update_captions(language, set_name, sample_index):
    dataset = dataset_dict[set_name]
    sample = dataset[int(sample_index)]
    caption_dict = sample["captions"][language]
    
    for key in [ "caption_alt_text_description",
                "caption_reference_description",
                "caption_title_and_reference_description",
                "context_page_description",
                "context_section_description",
                "hierarchical_section_title",
                "page_title",
                "section_title",
            ]:
        if key not in caption_dict:
            caption_dict[key] = "<Unavailable/>"
    
    
    return [gr.update(value=caption_dict["caption_alt_text_description"]), 
            gr.update(value=caption_dict["caption_reference_description"]),
            gr.update(value=caption_dict["caption_title_and_reference_description"]),
            gr.update(value=caption_dict["context_page_description"]),
            gr.update(value=caption_dict["context_section_description"]),
            gr.update(value=caption_dict["hierarchical_section_title"]),
            gr.update(value=caption_dict["page_title"]),
            gr.update(value=caption_dict["section_title"])]
def update_language_choices(set_name, sample_index):
    return gr.update(choices=list(dataset_dict[set_name][int(sample_index)]["captions"].keys()))
def load_sample(set_name, sample_index):
    # Load the dataset based on the set name (you'll need to implement this part)
    dataset = dataset_dict[set_name]

    # Retrieve the sample at the given index
    sample = dataset[int(sample_index)]

    # Extract the text, image, video, and audio from the sample (you'll need to adapt this to your specific dataset)
    languages = list(sample["captions"].keys())
    caption = sample["captions"]
    subtitles = sample["youtube_description_text"]
    wit_image = sample["wikipedia_caption_image"].permute(1, 2, 0).numpy()
    youtube_image = sample["youtube_random_video_sample_image"].permute(1, 2, 0).numpy()
    video = sample["youtube_content_video"].permute(0, 2, 3, 1).numpy() * 255
    audio = sample["youtube_content_audio"]
    
    video_path = f"temp_data/video-{set_name}-{sample_index}.mp4"
    audio_path = f"temp_data/audio-{set_name}-{sample_index}.mp3"
    
    if not pathlib.Path("temp_data/").exists():
        pathlib.Path("temp_data/").mkdir(parents=True, exist_ok=True)
    
    if not pathlib.Path(video_path).exists():
        torchvision.io.write_video(video_path, video, fps=25)
    if not pathlib.Path(audio_path).exists():
        torchaudio.save(audio_path, audio.unsqueeze(0), 16000)
    return "", "", "", "", "", "", "", "", subtitles, wit_image, youtube_image, video_path, audio_path


In [7]:

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # TALI Dataset Demo
    """)
    with gr.Row():
        with gr.Column():
            input_set_name = gr.Dropdown(["train", "val", "test"], label="Set name", info="Select the set to sample from")
        with gr.Column():
            input_sample_index = gr.Slider(minimum=0, maximum=100, step=1, interactive=True,
                label="Datapoint idx to sample",
                info="Select the idx to sample",
            )
    input_sample_index.change(update_length_options, input_set_name, input_sample_index)
    fetch_btn = gr.Button("Fetch sample")
    
    output_language = gr.Dropdown(label="Wiki language ID")
    with gr.Row():
        caption_alt_text_description = gr.Textbox(label="caption_alt_text_description")
        caption_reference_description = gr.Textbox(label="caption_reference_description")
        caption_title_and_reference_description = gr.Textbox(label="caption_title_and_reference_description")
        context_page_description = gr.Textbox(label="context_page_description")
        context_section_description = gr.Textbox(label="context_section_description")
        hierarchical_section_title = gr.Textbox(label="hierarchical_section_title")
        page_title = gr.Textbox(label="page_title")
        section_title = gr.Textbox(label="section_title")
    output_language.select(update_captions, [output_language, input_set_name, input_sample_index], [caption_alt_text_description, caption_reference_description, caption_title_and_reference_description, context_page_description, context_section_description, hierarchical_section_title, page_title, section_title])
    
    output_subtitle = gr.Text(label="Youtube Subtitles")
    with gr.Row():
        with gr.Column():
            output_wit_image = gr.Image(label="Wikipedia Image")
        with gr.Column():
            output_youtube_image = gr.Image(label="Youtube Image")
        with gr.Column():
            output_video = gr.Video(label="Youtube Video")
        with gr.Column():
            output_audio = gr.Audio(label="Youtube Audio")
    
    fetch_btn.click(update_language_choices, [input_set_name, input_sample_index], output_language)
    fetch_btn.click(fn=load_sample,
    inputs=[input_set_name, input_sample_index],
    outputs=[caption_alt_text_description,
            caption_reference_description,
            caption_title_and_reference_description,
            context_page_description,
            context_section_description,
            hierarchical_section_title,
            page_title,
            section_title,
            output_subtitle, output_wit_image, output_youtube_image, output_video, output_audio],)


In [8]:
demo.queue()
demo.launch(share=True, debug=True, enable_queue=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f5354eb614ae618570.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
